In [9]:
from keras.applications.inception_v3 import InceptionV3
import keras
from keras.layers import *
from keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

In [12]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = np.array([cv2.resize(img, (75, 75)) for img in X_train])
X_test = np.array([cv2.resize(img, (75, 75)) for img in X_test])

X_train = np.stack([X_train] * 3, axis=-1)
X_test = np.stack([X_test] * 3, axis=-1)

X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [13]:
base_model = InceptionV3(
    weights='imagenet',
    input_shape=(75,75, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


In [14]:
x = base_model.output
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
from keras.callbacks import EarlyStopping
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 32
epochs = 10

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 298s 131ms/step - accuracy: 0.8533 - loss: 0.4489
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 248s 132ms/step - accuracy: 0.9346 - loss: 0.2004
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 277s 147ms/step - accuracy: 0.9461 - loss: 0.1564
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 308s 164ms/step - accuracy: 0.9568 - loss: 0.1305
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 266s 141ms/step - accuracy: 0.9606 - loss: 0.1139
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 262s 139ms/step - accuracy: 0.9685 - loss: 0.0917
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 282s 150ms/step - accuracy: 0.9708 - loss: 0.0828
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 263s 140ms/step - accuracy: 0.9767 - loss: 0.0671
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 245s 130ms/step - accuracy: 0.9791 - loss: 0.0608
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 249s 133ms/step - accuracy: 0.9821 - loss: 0.0498


In [16]:
model.save('NN/model_mnist_inception.h5')

In [17]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 66ms/step - accuracy: 0.9410 - loss: 0.2297

Test accuracy: 0.946399986743927
